# 스레드 수준 지속성을 추가하는 방법 (함수형 API)

!!! info "필수 조건"

    이 가이드는 다음에 대한 익숙함을 가정합니다:
    
    - [함수형 API](../../concepts/functional_api/)
    - [지속성](../../concepts/persistence/)
    - [메모리](../../concepts/memory/)
    - [채팅 모델](https://python.langchain.com/docs/concepts/chat_models/)

!!! info "LangGraph API 사용자에게는 필요하지 않음"

    LangGraph API를 사용하는 경우 체크포인터를 수동으로 구현할 필요가 없습니다. API가 자동으로 체크포인팅을 처리합니다. 이 가이드는 자체 커스텀 서버에서 LangGraph를 구현할 때 관련이 있습니다.

많은 AI 애플리케이션은 동일한 [스레드](../../concepts/persistence#threads)에서 여러 상호 작용 간에 컨텍스트를 공유하기 위해 메모리가 필요합니다(예: 대화의 여러 턴). LangGraph 함수형 API에서 이러한 종류의 메모리는 [스레드 수준 지속성](https://langchain-ai.github.io/langgraph/concepts/persistence)을 사용하여 모든 [entrypoint()][langgraph.func.entrypoint] 워크플로우에 추가할 수 있습니다.

LangGraph 워크플로우를 생성할 때 [체크포인터](https://langchain-ai.github.io/langgraph/reference/checkpoints/#basecheckpointsaver)를 사용하여 결과를 유지하도록 설정할 수 있습니다:


1. 체크포인터의 인스턴스를 생성합니다:

    ```python
    from langgraph.checkpoint.memory import InMemorySaver
    
    checkpointer = InMemorySaver()       
    ```

2. `checkpointer` 인스턴스를 `entrypoint()` 데코레이터에 전달합니다:

    ```python
    from langgraph.func import entrypoint
    
    @entrypoint(checkpointer=checkpointer)
    def workflow(inputs)
        ...
    ```

3. 선택적으로 워크플로우 함수 시그니처에서 `previous` 매개변수를 노출합니다:

    ```python
    @entrypoint(checkpointer=checkpointer)
    def workflow(
        inputs,
        *,
        # 선택적으로 워크플로우 함수 시그니처에서 `previous`를 지정하여
        # 마지막 실행 시점의 워크플로우에서 반환된 값에 액세스할 수 있습니다
        previous
    ):
        previous = previous or []
        combined_inputs = previous + inputs
        result = do_something(combined_inputs)
        ...
    ```

4. 선택적으로 워크플로우에서 반환할 값과 체크포인터가 `previous`로 저장할 값을 선택합니다:

    ```python
    @entrypoint(checkpointer=checkpointer)
    def workflow(inputs, *, previous):
        ...
        result = do_something(...)
        return entrypoint.final(value=result, save=combine(inputs, result))
    ```

이 가이드는 워크플로우에 스레드 수준 지속성을 추가하는 방법을 보여줍니다.

!!! tip "참고"

    여러 대화 또는 사용자 간에 __공유__되는 메모리가 필요한 경우(교차 스레드 지속성) 이 [how-to 가이드](../cross-thread-persistence-functional)를 확인하세요.

!!! tip "참고"

    `StateGraph`에 스레드 수준 지속성을 추가해야 하는 경우 이 [how-to 가이드](../persistence)를 확인하세요.

## 설정

먼저 필요한 패키지를 설치해야 합니다

In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_anthropic

다음으로 Anthropic(우리가 사용할 LLM)에 대한 API 키를 설정해야 합니다.

In [ ]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("ANTHROPIC_API_KEY")

<div class="admonition tip">
    <p class="admonition-title">LangGraph 개발을 위한 <a href="https://smith.langchain.com">LangSmith</a> 설정</p>
    <p style="padding-top: 5px;">
        LangSmith에 가입하여 문제를 빠르게 발견하고 LangGraph 프로젝트의 성능을 개선하세요. LangSmith를 사용하면 LangGraph로 구축한 LLM 앱을 디버그, 테스트 및 모니터링하기 위해 추적 데이터를 사용할 수 있습니다 — 시작하는 방법에 대해 자세히 알아보려면 <a href="https://docs.smith.langchain.com">여기</a>를 참조하세요. 
    </p>
</div>

## 예제: 단기 메모리가 있는 간단한 챗봇

[채팅 모델](https://python.langchain.com/docs/concepts/chat_models/)을 호출하는 단일 작업이 있는 워크플로우를 사용할 것입니다.

먼저 사용할 모델을 정의해 봅시다:

In [3]:
from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(model="claude-3-5-sonnet-latest")

이제 작업과 워크플로우를 정의할 수 있습니다. 지속성을 추가하려면 [Checkpointer](https://langchain-ai.github.io/langgraph/reference/checkpoints/#langgraph.checkpoint.base.BaseCheckpointSaver)를 [entrypoint()][langgraph.func.entrypoint] 데코레이터에 전달해야 합니다.

In [ ]:
from langchain_core.messages import BaseMessage
from langgraph.graph import add_messages
from langgraph.func import entrypoint, task
from langgraph.checkpoint.memory import InMemorySaver


@task
def call_model(messages: list[BaseMessage]):
    response = model.invoke(messages)
    return response


checkpointer = InMemorySaver()


@entrypoint(checkpointer=checkpointer)
def workflow(inputs: list[BaseMessage], *, previous: list[BaseMessage]):
    if previous:
        inputs = add_messages(previous, inputs)

    response = call_model(inputs).result()
    return entrypoint.final(value=response, save=add_messages(inputs, response))

이 워크플로우를 사용하려고 하면 대화의 컨텍스트가 상호 작용 간에 유지됩니다:

!!! note 참고

    LangGraph Platform 또는 LangGraph Studio를 사용하는 경우 자동으로 수행되므로 entrypoint 데코레이터에 체크포인터를 전달할 __필요가 없습니다__.

이제 에이전트와 상호 작용하면 이전 메시지를 기억하는 것을 볼 수 있습니다!

In [5]:
config = {"configurable": {"thread_id": "1"}}
input_message = {"role": "user", "content": "hi! I'm bob"}
for chunk in workflow.stream([input_message], config, stream_mode="values"):
    chunk.pretty_print()

================================== Ai Message ==================================

Hi Bob! I'm Claude. Nice to meet you! How are you today?


항상 이전 스레드를 재개할 수 있습니다:

In [6]:
input_message = {"role": "user", "content": "what's my name?"}
for chunk in workflow.stream([input_message], config, stream_mode="values"):
    chunk.pretty_print()

================================== Ai Message ==================================

Your name is Bob.


새 대화를 시작하려면 다른 `thread_id`를 전달할 수 있습니다. 짠! 모든 메모리가 사라졌습니다!

In [7]:
input_message = {"role": "user", "content": "what's my name?"}
for chunk in workflow.stream(
    [input_message],
    {"configurable": {"thread_id": "2"}},
    stream_mode="values",
):
    chunk.pretty_print()

================================== Ai Message ==================================

I don't know your name unless you tell me. Each conversation I have starts fresh, so I don't have access to any previous interactions or personal information unless you share it with me.


!!! tip "토큰 스트리밍"

    챗봇에서 LLM 토큰을 스트리밍하려면 `stream_mode="messages"`를 사용할 수 있습니다. 자세히 알아보려면 이 [how-to 가이드](../streaming-tokens)를 확인하세요.